In [117]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torch.multiprocessing as mp

In [19]:
is_cuda = False
dev = torch.device('cpu')  
if torch.cuda.is_available():
    is_cuda = True
    dev = torch.device('cuda')  

In [3]:
N = 1000

In [4]:
x1 = np.random.normal(3, 2, N)
x2 = np.random.normal(0, 5, N)

X = np.vstack((x1, x2)).T

# Noise
n1 = np.random.normal(0, 2, N)
n2 = np.random.normal(0, 6, N)
n3 = np.random.normal(0, 10, N)

y1 = x1**2 + 5*x1*x2 + 3*np.abs(x2) + n1

In [59]:
X_train = torch.tensor(X[:800], requires_grad= True, device=dev)
y1_train = torch.tensor(y1[:800], requires_grad= True, device=dev)

X_test = torch.tensor(X[800:], requires_grad= True, device=dev)
y1_test = torch.tensor(y1[800:], requires_grad= True, device=dev)

In [ ]:
class VariationalLayer(nn.Module):
    def __init__(self, 
                 input_size, output_size,
                 prior_mu, prior_rho,
                 theta_mu_init, theta_rho_init
                ):
        super().__init__()
        
        # Bias weight
        input_size = input_size + 1
        
        # Defining Prior distribution (Gaussian)
        self.prior_mu = torch.tensor(prior_mu)
        self.prior_rho = torch.tensor(prior_rho)
        
        # Defining Variational class (Gaussian class)
        self.theta_mu = nn.Parameter(
            torch.Tensor(output_size, input_size).uniform_(-0.2, 0.2))
        self.theta_rho = nn.Parameter(
            torch.Tensor(output_size, input_size).uniform_(-2,-1))
        
        # Defining some constants
        self.logsqrttwopi = torch.log(
            torch.sqrt(2*torch.tensor(torch.pi)))
        self.K = torch.tensor(1)
    
    def rho_to_sigma(self, theta_rho):
        return torch.log(1 + torch.exp(theta_rho))

    def sample_weight(self, theta_mu, theta_rho):
        w = (theta_mu 
        + self.rho_to_sigma(theta_rho)*torch.randn(theta_mu.shape))
        return w

    def log_prob_gaussian(self, x, mu, rho):
            return (
                - self.logsqrttwopi
                - self.rho_to_sigma(rho)
                - ((x - mu)**2)/(2*self.rho_to_sigma(rho)**2)
            ).sum()
    
    def prior(self, w):
        return self.log_prob_gaussian(
            w, self.prior_mu, self.prior_rho)
        
    def variational(self, w, theta_mu, theta_rho):
        return self.log_prob_gaussian(
            w, theta_mu, theta_rho) 
    
    def forward(self, x_layer):
        theta_mu = self.theta_mu
        theta_rho = self.theta_rho
        w = self.sample_weight(theta_mu, theta_rho)
        

In [ ]:
class BayesianLinear(nn.Module):
    def __init__(self, layer_size, next_layer_size):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        # Weight parameters
        self.variat_mu = nn.Parameter(torch.Tensor(next_layer_size, layer_size+1).uniform_(-0.2, 0.2))
        self.variat_rho = nn.Parameter(torch.Tensor(next_layer_size, layer_size).uniform_(-2,-1))
        self.weight = VariationalInference(self.weight_mu, self.weight_rho)
        # Bias parameters
        self.bias_mu = nn.Parameter(torch.Tensor(out_features).uniform_(-0.2, 0.2))
        self.bias_rho = nn.Parameter(torch.Tensor(out_features).uniform_(-5,-4))
        self.bias = Gaussian(self.bias_mu, self.bias_rho)
        # Prior distributions
        self.weight_prior = ScaleMixtureGaussian(PI, SIGMA_1, SIGMA_2)
        self.bias_prior = ScaleMixtureGaussian(PI, SIGMA_1, SIGMA_2)
        self.log_prior = 0
        self.log_variational_posterior = 0

    def forward(self, x_layer, sample=False, calculate_log_probs=False):
        x_next_layer = 

        return F.linear(input, weight, bias)

In [128]:
class ANN(nn.Module):
    # Initialize the layers
    def __init__(self, input_size, output_size):
        super().__init__()
        
        self.act1 = nn.ReLU()
        self.linear1 = nn.Linear(input_size, 6).double()
        self.bn = nn.BatchNorm1d(6).double()
        self.linear2 = nn.Linear(6, output_size).double()
    
    # Perform the computation
    def forward(self, x):
        x = self.linear1(x)
        x = self.bn(x)
        x = self.act1(x)
        x = self.linear2(x)
        x = self.act1(x)
        return x
    
    def forward_parallel(self, x_samples):
        with mp.Pool(processes = 2) as p:
            results = p.map(self.forward, x_samples)
        return results

range(0, 8)

In [ ]:
import numpy as np
import torch
from torch.multiprocessing import Pool

X = np.array([[1, 3, 2, 3], [2, 3, 5, 6], [1, 2, 3, 4]])
X = torch.DoubleTensor(X).cuda()

def X_power_func(j):
    X_power = X**j
    return X_power

if __name__ == '__main__':
  with Pool(processes = 2) as p:   # Paralleizing over 2 GPUs
    results = p.map(X_power_func, range(8))

results

In [120]:
input_size = 2
output_size = 1

weight_mu = nn.Parameter(torch.Tensor(output_size, input_size).uniform_(-0.2, 0.2))
weight_rho = nn.Parameter(torch.Tensor(output_size, input_size).uniform_(-2,-1))

In [121]:
vi_obj = VariationalInference(0, 0.54, weight_mu, weight_rho)

In [129]:
h = ANN(input_size, output_size).to(dev)
opt_h = torch.optim.Adam(h.parameters(), lr=0.001)

mse_loss = nn.MSELoss(reduction='mean')

In [33]:
elbo_loss(xh, yh, h, w)

NameError: name 'elbo_loss' is not defined

In [55]:
w = vi_obj.sample_weight(weight_mu, weight_rho)
Q = vi_obj.variational(w, weight_mu, weight_rho)
P = vi_obj.variational(w)

index = np.random.randint(0, 800, 5)
x_h = X_train[index]
y_h = y1_train[index]
l = mse_loss(h(x_h), y_h)

tensor(-3.9606, grad_fn=<SumBackward0>)

In [70]:
h(x_h)

tensor([[0.0624],
        [0.0000],
        [0.0701],
        [0.2169],
        [0.1127]], dtype=torch.float64, grad_fn=<ReluBackward0>)

In [124]:
x_h.shape

torch.Size([200, 2])

In [130]:
h.forward_parallel(x_h)

RuntimeError: Cowardly refusing to serialize non-leaf tensor which requires_grad, since autograd does not support crossing process boundaries.  If you just want to transfer the data, call detach() on the tensor before serializing (e.g., putting it on the queue).

In [58]:
X_train

array([[ 1.41972735,  1.2029036 ],
       [ 3.64036384, -1.80001503],
       [ 1.66258872, -9.45797395],
       ...,
       [ 3.35859904,  2.77637386],
       [ 2.11101038,  5.22272453],
       [ 2.69671436,  2.78818971]])

In [ ]:
P = vi_obj.prior(vi_obj.sample_weight(weight_mu, weight_rho))

In [112]:
index = np.random.randint(0, 800, 200)
x_h = X_train[index]
y_h = y1_train[index]

In [113]:
x_h

tensor([[ 4.2213e+00, -6.2528e+00],
        [ 2.7539e+00,  2.2754e+00],
        [ 4.8483e+00, -3.6463e+00],
        [ 4.6008e+00, -2.2948e+00],
        [ 6.7030e+00, -2.3596e+00],
        [ 1.8571e+00,  6.8027e-01],
        [ 2.9115e+00, -5.4428e+00],
        [ 2.3574e+00, -4.6242e+00],
        [ 4.1867e+00, -5.0198e+00],
        [ 5.5665e+00,  1.7493e+00],
        [ 4.1867e+00, -5.0198e+00],
        [ 1.6467e+00, -1.4431e+00],
        [ 1.7152e+00, -2.6472e+00],
        [ 4.8483e+00, -3.6463e+00],
        [ 3.3220e+00,  3.2644e+00],
        [ 1.7996e+00, -6.0863e+00],
        [ 9.8054e-01,  3.7539e+00],
        [ 5.2619e+00, -2.2712e+00],
        [ 5.8081e+00,  4.0490e+00],
        [ 5.0169e+00,  3.7023e+00],
        [ 7.5814e-01,  3.0518e-01],
        [ 4.2604e+00,  5.6789e+00],
        [ 7.5471e+00, -1.5020e+00],
        [ 6.1520e+00,  4.9272e+00],
        [ 4.3850e+00,  2.8569e+00],
        [ 4.5348e+00,  3.5263e+00],
        [ 3.4467e+00, -4.1564e+00],
        [-1.3712e+00,  3.650

In [114]:
np.random.randint(0,N)

463

In [115]:
mse_loss(h(x_h), y_h)

tensor(8096.7533, dtype=torch.float64, grad_fn=<MseLossBackward0>)

In [174]:
w = sample_weight(weight_mu, weight_rho)

In [175]:
w

tensor([[-0.0181, -0.4754]], grad_fn=<AddBackward0>)

In [176]:
prior(w)

tensor(-3.9496, grad_fn=<SumBackward0>)

In [177]:
variational_posterior(w, weight_mu, weight_rho)

tensor(-4.1615, grad_fn=<SumBackward0>)